In [33]:
from glob import glob
import numpy as np
import pandas as pd
import plotly.express as px
import utils
import warnings

In [34]:
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

In [35]:
# Configuration
SCORE = "z-score"
TP_RANGE = 60  # 4 graphs per minute
FP_RANGE = 60  # 4 graphs per minute
DYNAMIC_FAULTS = ["node-drain", "pod-autoscaler", "pod-delete"]
STATIC_FAULTS = ["node-cpu-hog", "node-memory-hog", "pod-cpu-hog", "pod-memory-hog", "pod-network-latency"]
NODE_FAULTS = ["node-cpu-hog", "node-memory-hog", "node-drain"]

In [36]:
files = list(glob('../data/predictions/*.csv'))
dfs = []
for f in files:
    df_ = pd.read_csv(f)
    df_["timestamp"] = df_["timestamp"].apply(lambda x: pd.Timestamp(x))
    df_ = df_.set_index(["timestamp"], drop=True).sort_index()
    df_.rename(columns={SCORE: f.split("/")[-1].split(".")[0]}, inplace=True)
    dfs.append(df_)

In [37]:
df = pd.concat(dfs, axis=1)

In [38]:
df.shape

(10079, 2)

In [39]:
df.head()

,predictions\cluster,predictions\heroic_d3_adservice
timestamp,,
2024-02-13 06:00:10,False,False
2024-02-13 06:00:25,False,False
2024-02-13 06:00:40,False,False
2024-02-13 06:00:55,False,False
2024-02-13 06:01:10,False,False


In [40]:
df["sum"] = df.sum(axis=1)

In [41]:
df.head()
df.to_csv("../data/predictions/ensemble.csv")

In [42]:
labels = pd.read_csv("../data/labels_revised.csv")

In [43]:
labels["experiment_created_at"] = pd.to_datetime(labels["experiment_created_at"])
labels["experiment_started_at"] = pd.to_datetime(labels["experiment_started_at"])
labels["experiment_finished_at"] = pd.to_datetime(labels["experiment_finished_at"])

labels["started_at"] = pd.to_datetime(labels["started_at"])
labels["finished_at"] = pd.to_datetime(labels["finished_at"])

anomalies = pd.DataFrame()
for idx, row in labels.iterrows():
    if row['started_at'] >= df.index[0]:
        df_ = df.iloc[df.index.get_indexer([row['started_at']], method='nearest')]
        df_["fault_name"] = row["fault_name"]
        anomalies = pd.concat([anomalies, df_])

In [44]:
y = pd.Series([0 if idx not in anomalies.index else 1 for idx, row in df.iterrows()], index=df.index)
y_label = pd.Series(["normal" if idx not in anomalies.index else anomalies.loc[idx]["fault_name"] for idx, row in df.iterrows()], index=df.index)

In [45]:
Y_test = np.array([y for idx, y in enumerate(y) if df.index[idx] >= pd.Timestamp("2024-02-13 06:00:00")])
Y_test_ranges = []
for idx, label in enumerate(Y_test):
    if label == 1:
        Y_test_ranges.append((max(idx-TP_RANGE, 0), min(idx+TP_RANGE, len(Y_test)-1)))

In [46]:
y_pred = [idx for idx, i in enumerate(df["sum"].to_numpy()) if i == True]
precision, recall, f1 = utils.evaluate(Y_test_ranges, y_pred, FP_RANGE)

In [47]:
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1: {f1}")

Precision: 0.48148148148148145
Recall: 0.6842105263157895
F1: 0.5652173913043478


In [48]:
fig = px.line(df["sum"])

for idx, label in y.items():
    if label == 1:
        fault_name = y_label[idx]
        if fault_name in DYNAMIC_FAULTS:
            fig.add_vline(x=(idx.timestamp() - 3600) * 1000, opacity=0.5, line_color="green", annotation_text=fault_name)
        else:
            fig.add_vline(x=(idx.timestamp() - 3600) * 1000, opacity=0.5, line_color="red", annotation_text=fault_name)

if "annotations" in fig.to_dict()["layout"].keys():
    fig.update_layout(annotations=[{**a, **{"textangle": 90}} for a in fig.to_dict()["layout"]["annotations"]])
fig.show()